# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

In [29]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib
import copy

from sklearn.model_selection import train_test_split
from sklearn import tree, metrics
from sklearn.utils import shuffle
from catboost import CatBoostClassifier

### Задание 1 (3 балла)
Реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [84]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.n_estimators = n_estimators
        self.n_classes = 0
        self.X = None
        self.y = None
        self.trees = []
        self.tree_idx = []
        for _ in range(n_estimators):
            classifier = tree.DecisionTreeClassifier(
                criterion=criterion, 
                min_samples_leaf=min_samples_leaf,
                max_depth=max_depth,
                max_features=max_features
            )
            self.trees.append(classifier)
    
    def fit(self, X, y):
        self.n_classes = len(set(y))
        self.X = X
        self.y = y
        n_items = y.size
        for i in range(self.n_estimators):
            idx = np.random.choice(n_items, n_items)
            self.tree_idx.append(idx)
            X_temp, y_temp = X[idx], y[idx]
            self.trees[i].fit(X_temp, y_temp)
    
    def predict(self, X):
        predictions = np.zeros((X.shape[0], self.n_estimators), dtype=np.int64)
        for i in range(self.n_estimators):
            predictions[:, i] = self.trees[i].predict(X)
        counts = np.apply_along_axis(lambda x: np.bincount(x, minlength=self.n_classes), axis=1, arr=predictions)
        return np.apply_along_axis(lambda x: np.argmax(x), axis=1, arr=counts)

### Задание 3 (3 балла)
Оптимизируйте по AUC на кроссвалидации (размер валидационной выборки - 20%) параметры своей реализации Random Forest: 

максимальную глубину деревьев из [2, 3, 5, 7, 10], количество деревьев из [5, 10, 20, 30, 50, 100]. 

Постройте ROC кривую (и выведите AUC и accuracy) для лучшего варианта.

Подсказка: можно построить сразу 100 деревьев глубины 10, а потом убирать деревья и
глубину.

In [54]:
def std_scale(X):
    for col in list(X):
        mean = X[col].mean()
        std = X[col].std()
        X[col] = (X[col] - mean) / std
    return X

def read_cancer_dataset(path_to_csv):
    # Возвращает пару из X и y. X - массив векторов. y - соответствующие векторам метки
    df = shuffle(pd.read_csv(path_to_csv))
    y = (df['label'].values == "M").astype(int)
    X = df.drop('label', axis=1)
    X = std_scale(X).values
    return X, y

def read_spam_dataset(path_to_csv):
    # Возвращает пару из X и y. X - массив векторов. y - соответствующие векторам метки
    df = shuffle(pd.read_csv(path_to_csv))
    y = df['label'].values
    X = df.drop('label', axis=1)
    X = std_scale(X).values
    return X, y

def crossvalidation_train_test_split(X, y, ratio=0.2):
    # Возвращает X_train, y_train, X_test, y_test
    n_items = len(y)
    n = int(n_items * ratio)
    start, stop = 0, n
    while stop <= n_items:
        test_idx = np.arange(start, stop)
        yield (X[~test_idx], y[~test_idx], X[test_idx], y[test_idx])
        start, stop = stop, stop + n
    

In [79]:
X, y = read_cancer_dataset("hw1/cancer.csv")

auc_limit = 0.95

depth_choices = [2, 3, 5, 7, 10]
num_trees = [5, 10, 20, 30, 50, 100]

best_choice = 10, 100

for depth in reversed(depth_choices):
    for n_trees in reversed(num_trees):
        forest = RandomForestClassifier(max_depth=depth, n_estimators=n_trees)
        scores = []
        for X_train, y_train, X_test, y_test in crossvalidation_train_test_split(X, y):
            forest.fit(X_train, y_train)
            y_predict = forest.predict(X_test)
            score = metrics.roc_auc_score(y_test, y_predict)
            scores.append(score)
        total_score = sum(scores) / len(scores) 
        if total_score > auc_limit:
            best_choice = depth, n_trees
        else:
            break
        print(f'Для max_depth = {depth}, n_estimators = {n_trees} roc_auc score: {total_score}.')
print(f'Лучший выбор для max_depth, n_estimators = {best_choice}.')

Для max_depth = 10, n_estimators = 100 roc_auc score: 0.9508621164833662.
Для max_depth = 10, n_estimators = 50 roc_auc score: 0.9501725395437892.
Для max_depth = 10, n_estimators = 30 roc_auc score: 0.9509271805932327.
Для max_depth = 7, n_estimators = 100 roc_auc score: 0.9522314435569186.
Для max_depth = 7, n_estimators = 50 roc_auc score: 0.9520660401872899.
Для max_depth = 5, n_estimators = 100 roc_auc score: 0.9557433483188232.
Для max_depth = 5, n_estimators = 50 roc_auc score: 0.9554455779089327.
Лучший выбор для max_depth, n_estimators = (5, 50).


In [78]:
X, y = read_spam_dataset("hw1/spam.csv")

auc_limit = 0.92

depth_choices = [2, 3, 5, 7, 10]
num_trees = [5, 10, 20, 30, 50, 100]

best_choice = 10, 100
best_auc = 0

for depth in reversed(depth_choices):
    for n_trees in reversed(num_trees):
        forest = RandomForestClassifier(max_depth=depth, n_estimators=n_trees)
        scores = []
        for X_train, y_train, X_test, y_test in crossvalidation_train_test_split(X, y):
            forest.fit(X_train, y_train)
            y_predict = forest.predict(X_test)
            score = metrics.roc_auc_score(y_test, y_predict)
            scores.append(score)
        total_score = sum(scores) / len(scores) 
        if total_score > auc_limit:
            best_choice = depth, n_trees
        else:
            break
        print(f'Для max_depth = {depth}, n_estimators = {n_trees} roc_auc score: {total_score}.')
print(f'Лучший выбор для max_depth, n_estimators = {best_choice}.')

Для max_depth = 10, n_estimators = 100 roc_auc score: 0.9310182716819639.
Для max_depth = 10, n_estimators = 50 roc_auc score: 0.9295795034509986.
Для max_depth = 10, n_estimators = 30 roc_auc score: 0.9292286508060255.
Для max_depth = 10, n_estimators = 20 roc_auc score: 0.9268269512614408.
Для max_depth = 7, n_estimators = 100 roc_auc score: 0.9212141827236746.
Лучший выбор для max_depth, n_estimators = (7, 100).


### Задание 4 (3 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest выглядит так:
1. Посчитать out-of-bag ошибку предсказания `err_oob` (https://en.wikipedia.org/wiki/Out-of-bag_error)
2. Перемешать значения признака `j` у объектов выборки (у каждого из объектов изменится значение признака `j` на какой-то другой)
3. Посчитать out-of-bag ошибку (`err_oob_j`) еще раз.
4. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [113]:
def shuffle_feature(X, feature):
    X_temp = copy.deepcopy(X)
    t = copy.deepcopy(X[:, feature])
    np.random.shuffle(t)
    X_temp[:, feature] = t
    return X_temp


def feature_importance(rfc):
    err_oob = []
    err_oob_feature = {feature:[] for feature in range(rfc.X.shape[1])}
    predictions = np.zeros((rfc.y.size, rfc.n_estimators), dtype=np.int64)
    for i in range(rfc.n_estimators):
        idx = ~np.unique(rfc.tree_idx[i])
        y_true = rfc.y[idx]
        y_pred = rfc.predict(rfc.X[idx])
        err_oob.append(1 - np.sum(y_true == y_pred) / y_true.size)
        
        for feature in range(rfc.X.shape[1]):
            X = shuffle_feature(rfc.X, feature)
            y_pred = rfc.predict(X[idx])
            err_oob_feature[feature].append(1 - np.sum(y_true == y_pred) / y_true.size)
    
    err_oob = np.array(err_oob)
    
    imp = [np.mean(np.array(err_oob_feature[feature]) - err_oob)  for feature in range(rfc.X.shape[1])]
    return imp

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Протестируйте решение на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [114]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [0.0, 0.0, 0.19269400987902305, 0.19577865157868188, 0.44336303519507886, 0.0]


Проверьте, какие признаки важны для датасетов cancer и spam?

In [121]:
X, y = read_spam_dataset("hw1/spam.csv")
spam_feature_names = list(pd.read_csv("hw1/spam.csv"))[:-1]
rfc = RandomForestClassifier(max_depth=5, n_estimators=50)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
imp = feature_importance(rfc)
print("Importance:", imp)
most_important_features(imp, spam_feature_names, 10)

Accuracy: 0.9269723973049337
Importance: [0.0, 0.00017787327314991775, 0.0004259789379414558, -6.823609689525778e-06, 0.003930980977566076, 0.0009407792135339488, 0.01857384150825921, 0.0018499292162056124, 0.0002477615879583728, -3.46185232234042e-05, 0.0003662742508151995, 3.5154709732372335e-05, -6.896551724138167e-06, 0.00024006061185428252, 0.0, 0.008233544557439267, 0.0007086716179847219, 0.0008537557587013955, 0.0020538804235559647, 0.0009145804971579241, 0.0033617571218111063, -2.2720809420784249e-07, 0.0013893940762475876, -0.0005506538862923094, 0.013663416420035497, 0.0018779579726150497, 0.007817572947488347, 0.00026842471572644075, 0.0002478276544446967, 0.00013117822238095656, 1.3783667539075672e-05, 0.00011681694766104034, 2.0763789916187213e-05, 2.0602081553189322e-05, 0.00032308918949283563, 3.4257467708627145e-05, 0.0020982225876847904, 0.0, 0.00030946122929178757, 0.0, 0.0, 0.00035774615810545195, 8.22886741014428e-05, 1.3708251023538676e-05, 0.0006191037912214425, 0

array(['char_freq_!', 'word_freq_remove', 'char_freq_$', 'word_freq_hp',
       'capital_run_length_average', 'word_freq_free', 'word_freq_george',
       'capital_run_length_longest', 'capital_run_length_total',
       'word_freq_our'], dtype='<U26')

In [122]:
X, y = read_cancer_dataset("hw1/cancer.csv")
cancer_feature_names = list(pd.read_csv("hw1/cancer.csv"))[1:]
rfc = RandomForestClassifier(max_depth=5, n_estimators=50)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
imp = feature_importance(rfc)
print("Importance:", imp)
most_important_features(imp, cancer_feature_names, 10)

Accuracy: 0.9929701230228472
Importance: [0.0, 0.0030475444894176727, 0.0012804493364841551, -0.0003872875491544403, 0.0, 0.0, 0.001996671572054496, 0.004713879259517082, 0.00016653313949446468, 0.0007775880176099026, 0.0027756406644949805, 0.0, 0.0, 0.004110224727681131, 0.002000372718111347, 0.0, 0.0022289575092931035, 0.0016088339818197794, 0.0, 0.00010901205964474636, 0.0034943293092831486, 0.006278219860728899, 0.004225388142025401, 0.00598286685114092, 0.0017194876444462225, 0.00044357117578173175, 0.004614506711497022, 0.014042632016168068, 0.0007747630522579429, 0.0006647360791259471]


array(['28', '22', '24', '8', '27', '23', '14', '21', '2', '11'],
      dtype='<U2')

### Задание 5 (1 балл)
В качестве аьтернативы попробуем CatBoost. 

Туториалы можно найти, например, [здесь](https://catboost.ai/docs/) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb).

Также, как и реализованный ними RandomForest, примените его для наших датасетов.

In [64]:
X, y = read_spam_dataset("hw1/spam.csv")

cat_forest = CatBoostClassifier(logging_level='Silent')
scores = []
for X_train, y_train, X_test, y_test in crossvalidation_train_test_split(X, y):
    cat_forest.fit(X_train, y_train)
    y_predict = cat_forest.predict(X_test)
    score = metrics.roc_auc_score(y_test, y_predict)
    scores.append(score)
total_score = sum(scores) / len(scores) 
print(f'Для CatBoostClassifier c дефолтными параметрами roc_auc score: {total_score}.')

Для CatBoostClassifier c дефолтными параметрами roc_auc score: 0.9503852817701646.


In [65]:
X, y = read_cancer_dataset("hw1/cancer.csv")

cat_forest = CatBoostClassifier(logging_level='Silent')
scores = []
for X_train, y_train, X_test, y_test in crossvalidation_train_test_split(X, y):
    cat_forest.fit(X_train, y_train)
    y_predict = cat_forest.predict(X_test)
    score = metrics.roc_auc_score(y_test, y_predict)
    scores.append(score)
total_score = sum(scores) / len(scores) 
print(f'Для CatBoostClassifier c дефолтными параметрами roc_auc score: {total_score}.')

Для CatBoostClassifier c дефолтными параметрами roc_auc score: 0.9622937789997751.
